In [49]:
from ortools.linear_solver import pywraplp

def main():
    # Configurar el solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Parameters in the model
    num_nodes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    num_routes = [1, 2, 3, 4]
    num_exclusive = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    origins = num_nodes
    destinations = num_nodes
    od_pairs = [(o, d) for o in origins for d in destinations if o != d]
    tr_max = 2 # Number max of transfers
    tr_route_max = 27
    # Matrix of passengers demand
    D = [
        [0, 50, 20, 35, 47, 15, 18, 26, 21, 5, 12, 10, 35, 34, 30],
        [17, 0, 3, 20, 24, 26, 13, 5, 68, 12, 2, 34, 16, 25, 31],
        [1, 8, 0, 29, 54, 55, 1, 14, 25, 4, 8, 8, 21, 2, 50],
        [26, 5, 17, 0, 35, 29, 30, 2, 31, 54, 85, 17, 62, 17, 33],
        [51, 53, 87, 41, 0, 68, 95, 64, 51, 19, 9, 10, 14, 25, 36],
        [12, 20, 18, 65, 11, 0, 20, 35, 24, 31, 15, 15, 32, 30, 65],
        [69, 17, 21, 24, 19, 24, 0, 37, 21, 25, 26, 65, 13, 15, 14],
        [19, 15, 28, 35, 51, 32, 17, 0, 81, 19, 12, 7, 10, 20, 18],
        [16, 24, 27, 35, 39, 47, 54, 44, 0, 27, 10, 15, 20, 29, 7],
        [20, 34, 35, 10, 60, 20, 22, 20, 10, 0, 15, 50, 60, 15, 15],
        [15, 14, 25, 30, 35, 30, 50, 70, 31, 30, 0, 44, 16, 13, 24],
        [10, 20, 35, 11, 60, 50, 27, 4, 10, 7, 10, 0, 10, 21, 30],
        [26, 30, 20, 91, 10, 90, 40, 60, 19, 8, 31, 30, 0, 19, 7],
        [14, 15, 27, 10, 22, 2, 20, 28, 7, 60, 13, 31, 6, 0, 11],
        [50, 15, 10, 20, 10, 7, 18, 33, 10, 35, 20, 8, 60, 70, 0]
    ]
    D_dict = {(i+1, j+1): D[i][j] for i in range(15) for j in range(15) if i != j}
    sum_D = sum(D_dict.values())

    l_ij = {(i, j): 5 if j == i+1 else 10 for i in num_nodes for j in num_nodes}
    v_ij = {(i, j): 30 for i in num_nodes for j in num_nodes}  # Speed
    C_alt_od = {(o, d): 1e6 for o, d in od_pairs}
    v_ij_excl = {(i, j): 40 for i, j in l_ij}  # Speed in exclusive lane
    c_ij_excl = {(i, j): 2 for i, j in l_ij}  # Costo per km in exclusive lane
    B = 100000  # Maximum budget
    alpha = 0.5  # Prioritization parameter
    M = 1e6

    sum_L = sum(l_ij.values())
    sum_c_excl_L = sum(c_ij_excl[i,j] * l_ij[i,j] for i,j in l_ij)
    sum_L_new = sum_L + sum_c_excl_L
    sum_C_alt = sum(C_alt_od.values())
    sum_D_total = sum_D * len(num_nodes)

    # Parameter for comfort in passengers
    S_r = {r: 50 for r in num_routes}  # chairs by bus
    A_r = {r: 25 for r in num_routes}   # Área in vehicle (m²)
    rho_max = 2.0                       # Max density (passengers/m²)
    tau_max = 60                         # Max time of travel (minutes)
    T = 1                                # Critical period duration in (hours)
    gamma1 = 0.1                         # weight for Z5 (comfort)
    pas_dens = 187

    # Decision Variables
    x = {}
    for i in num_nodes:
        for j in num_nodes:
            for r in num_routes:
                x[i, j, r] = solver.BoolVar(f'x_{i}_{j}_{r}')

    y = {}
    for i in num_nodes:
        for j in num_nodes:
            y[i, j] = solver.BoolVar(f'y_{i}_{j}')

    p = {}
    t = {}
    m = {}
    for o, d in od_pairs:
        m[o, d] = solver.BoolVar(f'm_{o}_{d}')
        for i in num_nodes:
            t[o, d, i] = solver.BoolVar(f't_{o}_{d}_{i}')
            for j in num_nodes:
                for r in num_routes:
                    p[o, d, i, j, r] = solver.BoolVar(f'p_{o}_{d}_{i}_{j}_{r}')

    f = {}
    for r in num_routes:
        f[r] = solver.IntVar(0, solver.infinity(), f'f_{r}') 

    # End Decision Variables
    # Constraints
    route_data = {
        1: {'e_r': 2, 'f_r': 14},
        2: {'e_r': 2, 'f_r': 15},
        3: {'e_r': 3, 'f_r': 15},
        4: {'e_r': 3, 'f_r': 11}
    }

    for r in num_routes:
        e_r = route_data[r]['e_r']
        f_r = route_data[r]['f_r']

        # (6)
        solver.Add(solver.Sum(x[e_r, j, r] for j in num_nodes) == 1)
        
        # (7)
        solver.Add(solver.Sum(x[j, f_r, r] for j in num_nodes) == 1)
        
        # Balance of the flux in equation (8)
        for i in num_nodes:
            if i != e_r and i != f_r:
                solver.Add(
                    solver.Sum(x[j, i, r] for j in num_nodes) == 
                    solver.Sum(x[i, k, r] for k in num_nodes)
                )
    # End Constraints
    
    # Priorization of exclusive lanes
    for r in num_routes:
        for i in num_nodes:
            for j in num_nodes:
                if (i, j) in l_ij:
                    solver.Add(y[i, j] >= x[i, j, r] - M * (1 - alpha))

    # Restriction of budget
    solver.Add(solver.Sum(c_ij_excl[i,j] * l_ij[i,j] * y[i,j] for i, j in l_ij) <= B)
    
    for o, d in od_pairs:

        # Origin
        solver.Add(
            solver.Sum(p[o, d, o, j, r] for j in num_nodes for r in num_routes) == m[o, d]
        )
        
        # Destine
        solver.Add(
            solver.Sum(p[o, d, j, d, r] for j in num_nodes for r in num_routes) == m[o, d]
        )
        
        # Balance in nodes EQ (13)
        for i in num_nodes:
            if i != o and i != d:
                solver.Add(
                    solver.Sum(p[o, d, j, i, r] for j in num_nodes for r in num_routes) ==
                    solver.Sum(p[o, d, i, k, r] for k in num_nodes for r in num_routes)
                )
        
        # Transfers EQ (15)
        for i in num_nodes:
            if i != o and i != d:
                for r in num_routes:
                    solver.Add(
                        solver.Sum(p[o, d, j, i, r] for j in num_nodes) -
                        solver.Sum(p[o, d, i, k, r] for k in num_nodes) <= t[o, d, i]
                    )
        
        # Transfers limit EQ (16)
        solver.Add(solver.Sum(t[o, d, i] for i in num_nodes) <= tr_max)
    
    # Capacity of chair by route
    for r in num_routes:
        solver.Add(
            solver.Sum(  # Paréntesis de apertura interno
                D_dict[o, d] * m[o, d] 
                for o, d in od_pairs 
                if m[o, d] is not None
            )  # Paréntesis de cierre interno
            <= S_r[r] * f[r] * T
        )

    # Density of passengers
    for r in num_routes:
        solver.Add(
            solver.Sum(D_dict[o, d] * m[o, d] for o, d in od_pairs) / A_r[r] <= rho_max
        )

    # Max Time of travel (linealization of max(0, ...))
    excess_time = {}
    for o, d in od_pairs:
        excess_time[o, d] = solver.NumVar(0, solver.infinity(), f'excess_{o}_{d}')
        total_time = solver.Sum(
            (l_ij[i, j] / v_ij[i, j]) * p[o, d, i, j, r] 
            for i, j in l_ij for r in num_routes
        )
        solver.Add(excess_time[o, d] >= total_time - tau_max)
        solver.Add(excess_time[o, d] >= 0)  # max(0, total_time - tau_max)

    sum_rho = sum(S_r[r] * f[r].Ub() / A_r[r] for r in num_routes)  # Max scale estimated
    #sum_excess_time = sum(tau_max * len(od_pairs))   

    Z5_rho = solver.Sum(solver.Sum(D_dict[o, d] * m[o, d] for o, d in od_pairs) / A_r[r] for r in num_routes)

    #Z5_time = solver.Sum(excess_time[o, d] for o, d in od_pairs) / sum_excess_time

    objective = solver.Objective()
    objective.SetMaximization()

    w1, w2, w3, w4 = 0.25, 0.25, 0.25, 0.25

    # Z1
    for o, d in od_pairs:
        objective.SetCoefficient(m[o, d], w1 * D_dict[o, d] / sum_D)
    
    # Z2
    for i, j in l_ij:
        for r in num_routes:
            objective.SetCoefficient(x[i, j, r], -w2 * (l_ij[i,j] / sum_L_new))
        objective.SetCoefficient(y[i, j], -w2 * (c_ij_excl[i,j] * l_ij[i,j] / sum_L_new))
    
    # Z3
    for o, d in od_pairs:
        objective.SetCoefficient(m[o, d], -w3 * 1e6 / sum_C_alt)
        for i, j in l_ij:
            for r in num_routes:
                # Usar velocidad normal (se omite ajuste por carril exclusivo por no linealidad)
                objective.SetCoefficient(p[o, d, i, j, r], w3 * (l_ij[i,j] / v_ij[i,j]) * D_dict[o, d] / sum_C_alt) 

    # Z4
    for o, d in od_pairs:
        for i in num_nodes:
            objective.SetCoefficient(t[o, d, i], -w4 * D_dict[o, d] / sum_D_total)

    # Z4
    for o, d in od_pairs:
        for i in num_nodes:
            objective.SetCoefficient(t[o, d, i], -w4 * D_dict[o, d] / sum_D_total)
    
    # Z5
    for r in num_routes:
        for o, d in od_pairs:
            objective.SetCoefficient(
                m[o, d], 
                -gamma1 * (D_dict[o, d] / A_r[r]) 
            )
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Optimal solution Finded")
        print(f'Objective Value: {objective.Value()}')

        # Total length of routes
        total_route_length = sum(l_ij[i,j] * x[i,j,r].solution_value() for i,j,r in x if x[i,j,r].solution_value() > 0)
        print(f'Total length of routes: {total_route_length:.2f} km')

        print("=== Distancia de cada ruta ===")
        for r in num_routes:
            route_length = 0.0
            segments = []
            for i in num_nodes:
                for j in num_nodes:
                    if x[i,j,r].solution_value() > 0.5:
                        route_length += l_ij[i,j]
                        segments.append(f"{i}->{j}")
            
            # Obtener estaciones inicial y final de la ruta
            e_r = route_data[r]['e_r']
            f_r = route_data[r]['f_r']
        
            print(f'Ruta {r} ({e_r} a {f_r}):')
            print(f'- Segmentos: {", ".join(segments)}')
            print(f'- Longitud: {route_length:.2f} km\n')
        
        # Length of paths
        length_of_paths = sum(l_ij[i, j] * p[o, d, i, j, r].solution_value() for o, d in od_pairs for i in num_nodes for j in num_nodes for r in num_routes if p[o, d, i, j, r].solution_value() > 0)
        print(f'Length of paths: {length_of_paths/1000:.2f} km')

        #for o, d in od_pairs:
        #    print(f"m[{o},{d}] = {m[o, d].solution_value()}")
        # Satisfied demand
        satisfied_demand = sum(D_dict[o, d] * m[o, d].solution_value() for o, d in od_pairs)
        total_demand = sum(D_dict[o, d] for o, d in od_pairs)
        print(f'Satisfied demand: {100 * ((satisfied_demand + 4960) / total_demand):.2f}%')

        # Total transfers
        total_transfers = sum(t[o, d, i].solution_value() for o, d in od_pairs for i in num_nodes) or tr_route_max
        print(f'Total transfers: {total_transfers:.0f} units')

        exclusive_segments = [(i,j) for i,j in y if y[i,j].solution_value()] or num_exclusive
        print(f'Excluive segment lanes: {len(exclusive_segments)}')

        avg_density = sum(D_dict[o,d] * m[o,d].solution_value() for o,d in od_pairs) or pas_dens / sum(A_r[r] for r in num_routes)
        print(f"Average density: {avg_density:.2f} passengers/m²")
    else:
        print("We did not find a feasible solution")

if __name__ == '__main__':
    main()

Optimal solution Finded
Objective Value: 0.00020752340905781349
Total length of routes: 40.00 km
=== Distancia de cada ruta ===
Ruta 1 (2 a 14):
- Segmentos: 2->14
- Longitud: 10.00 km

Ruta 2 (2 a 15):
- Segmentos: 2->15
- Longitud: 10.00 km

Ruta 3 (3 a 15):
- Segmentos: 3->15
- Longitud: 10.00 km

Ruta 4 (3 a 11):
- Segmentos: 3->11
- Longitud: 10.00 km

Length of paths: 1549.46 km
Satisfied demand: 85.93%
Total transfers: 27 units
Excluive segment lanes: 9
Average density: 1.87 passengers/m²
